# QUICKSTART
ML에 있어서 자주 등장하는 task를 실행해보자.
각 섹션을 깊게 알아보는 것은, 앞으로 할 일이다.

## Working with data
데이터 다루는 원시 타입은 2가지이다. 
- `torch.utils.data.DataLoader`
- `torch.utils.data.Dataset`

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

### domain-specific libs
파이토치는 domain-specific libraries를 제공한다
- TorchText
- TorchVision
- TorchAudio
각각 데이터셋도 가지고 있다.

In [3]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

우리는 `DataLoader`에 `Dataset`을 arg로 넘겨줄 것이다.
이것은 데이터셋을 iterable로 래핑해준다.
그리고 다음과 같은 것들을 지원한다.
- automatic batching
- sampling
- shuffling
- multiprocess data loading

In [4]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


## Creating Models
- `nn.Module`을 상속하여 신경망 클래스를 만든다.
- `__init__` 함수에서 layers을 정의한다.
- `forward()` 에서 데이터 전달 방법을 정의한다.

In [5]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


## Optimizing the Model Parameters
모델 학습을 위해서 2가지가 필요하다
- `loss function`
- `optimizer`

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

학습 loop 한번 마다,
모델은 training dataset에 예측을 수행한다.
그리고 예측에러를 역전파하여 모델 파라미터를 개선시킨다

In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

우리는 test dataset을 통해서 학습이 잘 되었는지 모델 퍼포먼스도 체크해야한다

In [8]:
def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

학습 과정은 몇번의 iteration으로 실행된다(epochs).  
각 Epoch 동안, 모델은 더 나은 예측을 만들어낼 수 있는 파라미터를 학습한다.  
매 epoch 마다 모델의 accuracy와 loss를 출력하자;  
매 epoch 마다 accuracy가 증가하고, loss가 감소하는 것을 확인하길 원한다

In [9]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done!")

Epoch 1
-------------------------------
loss: 2.300197  [    0/60000]
loss: 2.295542  [ 6400/60000]
loss: 2.284127  [12800/60000]
loss: 2.286937  [19200/60000]
loss: 2.280819  [25600/60000]
loss: 2.254400  [32000/60000]
loss: 2.267013  [38400/60000]
loss: 2.239463  [44800/60000]
loss: 2.232718  [51200/60000]
loss: 2.214593  [57600/60000]
Test Error: 
 Accuracy: 34.4%, Avg loss: 0.034966 

Epoch 2
-------------------------------
loss: 2.213126  [    0/60000]
loss: 2.226512  [ 6400/60000]
loss: 2.182720  [12800/60000]
loss: 2.214651  [19200/60000]
loss: 2.194373  [25600/60000]
loss: 2.120463  [32000/60000]
loss: 2.171832  [38400/60000]
loss: 2.112094  [44800/60000]
loss: 2.117056  [51200/60000]
loss: 2.067624  [57600/60000]
Test Error: 
 Accuracy: 36.0%, Avg loss: 0.033101 

Epoch 3
-------------------------------
loss: 2.079744  [    0/60000]
loss: 2.111070  [ 6400/60000]
loss: 2.024653  [12800/60000]
loss: 2.102195  [19200/60000]
loss: 2.050222  [25600/60000]
loss: 1.920347  [32000/600

## Saving Models
모델을 저장하는 범용적인 방법은  
내부 상태 사전 자료형을 serialize하는 것이다(모델 파라미터를 포함한).

In [10]:
torch.save(model.state_dict(), "model.pth") # pth 파일로 serialize 하고 있음
print("Saved Pytorch Model State to model.pth")

Saved Pytorch Model State to model.pth


## Loading Models
모델을 불러오는 과정은  
모델 구조를 새로 생성하고, 상태 사전 자료형을 그것에 불러오는 것을 포함한다.

In [12]:
model = NeuralNetwork() # 기존 모델 구조 클래스로 새로운 인스턴스를 생성. 이러면 pth 외에 소스 코드도 필요하겠네..!
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

이제 모델을 불러왔으니 예측에 사용할 수 있다

In [14]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1] # feature와 label
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y] # one hot vector에서 argmax로 분류 결과값 찾음
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
